In [1]:
import higlass as hg
import numpy as np
import pandas as pd
import bioframe
import cooltools
import cooler

import utils as utils

### creating ctcf table

In [2]:
###importing ctcf peaks ####
ctcf_path='/project/fudenber_735/collaborations/karissa_2022/2022_09_features_for_RNAseq/ChIP-seq_in_WT-parental-E14'
ctcf = bioframe.read_table(ctcf_path+'/CTCF_peaks_called_on_4reps_foundInatLeast2reps_noBlacklist.bed', schema='bed')#.query(f'chrom in {clr.chromnames}')

### importing ctcf motifs ###
direct='/project/fudenber_735/motifs/mm10/jaspar/MA0139.1.tsv.gz'
motif=bioframe.read_table(direct)
motif=motif.rename(columns={0: 'chrom', 1: 'start',2:'end',3:'name',4:'score',5:'pval',6:'strand'})

### overlap between peaks and motifs### 
peaks_motifs = bioframe.overlap(ctcf,motif,how='inner')
peaks_motifs=peaks_motifs.rename(columns={'strand':'strand_','strand_':'strand'})
peaks_motifs=peaks_motifs[['chrom','start','end','strand']]


In [ ]:
import matplotlib.pylab as plt
peaks_motifs = bioframe.overlap(ctcf,motif, how='inner')
for i, reg_pair in peaks_motifs.iterrows():
    bioframe.vis.plot_intervals_arr(
        starts = [reg_pair.start,reg_pair.start_],
        ends = [reg_pair.end,reg_pair.end_],
        colors = ['skyblue', 'lightpink'],
        levels = [2,1],
        xlim = (0,16),
        show_coords = True)
    plt.title(f'overlapping pair #{i}')

/home1/rahmanin/.local/lib/python3.11/site-packages/bioframe/vis.py:122: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=figsize)


In [3]:
region_ctcf_motif = utils.region_data_frame(peaks_motifs, region = 'chr1:34000000-44000000')
region_ctcf_motif

/home1/rahmanin/start/polychrom/projects/Site_wise_occupancy/utils.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  region_dataframe['mid']=(region_dataframe.end+region_dataframe.start)/2
/home1/rahmanin/start/polychrom/projects/Site_wise_occupancy/utils.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  region_dataframe['lattice_loc'] = ((region_dataframe['mid'] - region_start) // lattice_size).astype('int')


,chrom,start,end,strand,mid,lattice_loc
0,chr1,34005641,34006016,+,34005828.5,23
1,chr1,34011003,34011576,+,34011289.5,45
2,chr1,34049624,34049965,-,34049794.5,199
3,chr1,34068201,34068742,-,34068471.5,273
4,chr1,34093645,34094093,+,34093869.0,375
...,...,...,...,...,...,...
268,chr1,43738188,43738540,-,43738364.0,38953
269,chr1,43760145,43760509,+,43760327.0,39041
270,chr1,43762617,43762972,-,43762794.5,39051
271,chr1,43898949,43899413,+,43899181.0,39596


In [4]:
df = region_ctcf_motif
repetitive_values = df['lattice_loc'][df['lattice_loc'].duplicated()]

unique_values = df['lattice_loc'].unique()


In [8]:
region_ctcf_motif.to_csv('chr1_34mb_44mb.csv',index=False)

In [17]:
new_region = 'chr1:34000000-35000000'
bioframe.parse_region_string(region)

('chr1', 34000000, 44000000)

### processing for simulaiton

In [3]:
input_file = 'data/chr1_34mb_44mb_with_predicted_occupancy.csv'
df = pd.read_csv(input_file)
df = df.query('lattice_loc<=10000')
df

,chrom,start,end,mid,strand,lattice_loc,predicted_occupancy
0,chr1,34005641,34006016,34005828.5,+,23,0.089047
1,chr1,34011003,34011576,34011289.5,+,45,0.220059
2,chr1,34049624,34049965,34049794.5,-,199,0.260436
3,chr1,34068201,34068742,34068471.5,-,273,0.644597
4,chr1,34093645,34094093,34093869.0,+,375,0.197648
...,...,...,...,...,...,...,...
63,chr1,36321730,36322172,36321951.0,-,9287,0.180413
64,chr1,36324025,36324431,36324228.0,+,9296,0.257837
65,chr1,36454412,36454833,36454622.5,+,9818,0.277825
66,chr1,36471503,36472006,36471754.5,-,9887,0.090944


In [4]:
dataframe = utils.make_region_occupancy(input_file)
target_dataframe = dataframe.query('lattice_loc<=10000')
target_dataframe

,chrom,start,end,mid,strand,lattice_loc,predicted_occupancy
0,chr1,34005641,34006016,34005828.5,+,23,0.089047
1,chr1,34011003,34011576,34011289.5,+,45,0.220059
2,chr1,34049624,34049965,34049794.5,-,199,0.260436
3,chr1,34068201,34068742,34068471.5,-,273,0.644597
4,chr1,34093645,34094093,34093869.0,+,375,0.197648
...,...,...,...,...,...,...,...
234,chr1,43738188,43738540,43738364.0,-,38953,0.254767
235,chr1,43760145,43760509,43760327.0,+,39041,0.337179
236,chr1,43762617,43762972,43762794.5,-,39051,0.163237
237,chr1,43898949,43899413,43899181.0,+,39596,0.288898


In [11]:
target_dataframe

,chrom,start,end,mid,strand,lattice_loc,predicted_occupancy
0,chr1,34005641,34006016,34005828.5,+,23,0.089047
1,chr1,34011003,34011576,34011289.5,+,45,0.220059
2,chr1,34049624,34049965,34049794.5,-,199,0.260436
3,chr1,34068201,34068742,34068471.5,-,273,0.644597
4,chr1,34093645,34094093,34093869.0,+,375,0.197648
...,...,...,...,...,...,...,...
56,chr1,36321730,36322172,36321951.0,-,9287,0.180413
57,chr1,36324025,36324431,36324228.0,+,9296,0.257837
58,chr1,36454412,36454833,36454622.5,+,9818,0.277825
59,chr1,36471503,36472006,36471754.5,-,9887,0.090944


### making ctcf sites

In [13]:
ctcfleftposition = list(target_dataframe[target_dataframe['strand']=='+']['lattice_loc'])
ctcfrightposition = list(target_dataframe[target_dataframe['strand']=='-']['lattice_loc'])
print(ctcfleftposition,ctcfrightposition)

[23, 45, 375, 760, 806, 874, 1852, 1944, 2045, 2053, 2165, 2336, 2570, 2954, 3106, 3167, 3469, 3567, 4131, 5067, 5553, 5570, 6907, 7063, 7358, 8581, 8899, 9025, 9296, 9818] [199, 273, 659, 1164, 1640, 1944, 2064, 2336, 2410, 2803, 3102, 3469, 3588, 3648, 5067, 5542, 5861, 5913, 6864, 7387, 8041, 8260, 8539, 8598, 9012, 9019, 9109, 9117, 9287, 9887, 9970]


In [15]:
typedict = {'strong_CTCF':1, 'weak_CTCF':0}
V = typedict['strong_CTCF']